# **ISSCC 2026 Code-a-Chip Challenge**
# **LLMForge: Joint HW/SW Optimization of Edge-Scale Transformer Architecture**
## Author: Xinting Jiang, Minxing Chu - Brown University 

xinting_jiang@brown.edu, ReaLLMASIC Lab

Please feel free to reach out!

# Overview
This notebook documents our end-to-end workflow for transformer hardware/software co-design for the ISSCC 2026 Code-a-Chip Challenge. The exploration is motivated by the need to deliver competitive language models on resource-constrained edge platforms, where accuracy gains must be balanced against strict latency, energy, and silicon budgets.

We proceed in six stages:
- formalize the GPT-2–style transformer search space and expose interactive controls for key architectural levers.
- quantify the software-only configuration space before extending it to hardware-aware combinations.
- map candidate transformer layers onto the custom accelerator while enforcing feasibility constraints from the systolic array and SRAM fabric;
- sweep feasible HW/SW pairs to collect performance, power, and area metrics suitable for downstream analytics.
- visualize Pareto trade-offs across latency, energy, and accuracy to surface promising design points.
- motivate heterogeneous transformers and Infinite Attention as forward-looking avenues that further expand the co-design space.

Github repo used: `https://github.com/ReaLLMASIC/ReaLLM-Forge.git`

## Environment Setup

### Installing Yosys, Openroad, sky130pdk

In [ ]:
import os
import pathlib
import sys

!pip install matplotlib pandas pyinstaller numpy
!apt-get install -y ruby-full time build-essential
!apt install -f libqt4-designer libqt4-xml libqt4-sql libqt4-network libqtcore4 libqtgui4
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))
CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel litex-hub \
                        --channel main \
                        open_pdks.sky130a \
                        openroad \
                        yosys
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel conda-forge \
                        tcllib gdstk pyyaml click svgutils ngspice

## Transformer Architecture Design Space

Considering the gpt2-like decoder-only transformer structure, it would have the follow parameter config variations.

![Transformer Architecture](images/transformer_sw.png)

| Symbol | Parameter | Description | 
|:--:|:--|:--|
| $\text{n\_embd}$ | Model dimension | Hidden size of layer $\ell$ |
| $\text{n\_head}$ | Attention heads | Number of parallel attention heads | 
| $\text{mlp\_ratio}$ | MLP expansion ratio | Expansion ratio in feedforward block | 
| $\text{block\_size}$ | Block size | Number of visible tokens per query | 
| $\text{n\_layers}$ | Layers | Number of Layer |

Consider a homogeneous architecture (i.e. each layer shares the same setting), the search space could be configured in the following block

In [ ]:
# Interactive search-space controls
!pip install ipywidgets
from ipywidgets import widgets
from IPython.display import display, clear_output


def show_config(max_n_layers,
                n_embd_min, n_embd_max, n_embd_step,
                mlp_min, mlp_max, mlp_step,
                bs_min, bs_max, bs_step,
                max_n_heads):
    print(f"Maximum number of layers: {max_n_layers}")
    print(f"Embedding dimension range: min={n_embd_min}, max={n_embd_max}, step={n_embd_step}")
    print(f"MLP ratio range: min={mlp_min}, max={mlp_max}, step={mlp_step}")
    print(f"Block size range: min={bs_min}, max={bs_max}, step={bs_step}")
    print(f"Maximum number of heads: {max_n_heads}")

    # Basic validation feedback
    errs = []
    if n_embd_step <= 0:
        errs.append("Embedding step must be > 0")
    if n_embd_min > n_embd_max:
        errs.append("Embedding min cannot be greater than max")

    if mlp_step <= 0:
        errs.append("MLP ratio step must be > 0")
    if mlp_min > mlp_max:
        errs.append("MLP ratio min cannot be greater than max")

    if bs_step <= 0:
        errs.append("Block size step must be > 0")
    if bs_min > bs_max:
        errs.append("Block size min cannot be greater than max")

    if errs:
        print("\nIssues:")
        for e in errs:
            print(" -", e)


# Wider labels so full names are visible
label_style = {"description_width": "initial"}

# Widgets with full names
w_max_n_layers = widgets.IntSlider(
    value=12, min=1, max=64, step=1,
    description="Max n_layers",
    style=label_style,
)

# Embedding (d_model) range
w_n_embd_min = widgets.IntText(
    value=256, description="Embedding dimension (min)",
    style=label_style,
)
w_n_embd_max = widgets.IntText(
    value=768, description="Embedding dimension (max)",
    style=label_style,
)
w_n_embd_step = widgets.IntText(
    value=64, description="Embedding dimension (step)",
    style=label_style,
)

# MLP ratio range
w_mlp_min = widgets.IntText(
    value=1, description="MLP ratio (min)",
    style=label_style,
)
w_mlp_max = widgets.IntText(
    value=8, description="MLP ratio (max)",
    style=label_style,
)
w_mlp_step = widgets.IntText(
    value=1, description="MLP ratio (step)",
    style=label_style,
)

# Block size range
w_bs_min = widgets.IntText(
    value=256, description="Block size (min)",
    style=label_style,
)
w_bs_max = widgets.IntText(
    value=2048, description="Block size (max)",
    style=label_style,
)
w_bs_step = widgets.IntText(
    value=64, description="Block size (step)",
    style=label_style,
)

# Heads
w_max_n_heads = widgets.IntSlider(
    value=8, min=1, max=64, step=1,
    description="Max n_heads",
    style=label_style,
)

ui = widgets.VBox([
    w_max_n_layers,
    widgets.HBox([w_n_embd_min, w_n_embd_max, w_n_embd_step]),
    widgets.HBox([w_mlp_min, w_mlp_max, w_mlp_step]),
    widgets.HBox([w_bs_min, w_bs_max, w_bs_step]),
    w_max_n_heads,
])

out = widgets.Output()


def _refresh_output(*_):
    with out:
        clear_output(wait=True)
        show_config(
            w_max_n_layers.value,
            w_n_embd_min.value, w_n_embd_max.value, w_n_embd_step.value,
            w_mlp_min.value, w_mlp_max.value, w_mlp_step.value,
            w_bs_min.value, w_bs_max.value, w_bs_step.value,
            w_max_n_heads.value,
        )


for _w in [
    w_max_n_layers,
    w_n_embd_min, w_n_embd_max, w_n_embd_step,
    w_mlp_min, w_mlp_max, w_mlp_step,
    w_bs_min, w_bs_max, w_bs_step,
    w_max_n_heads,
]:
    _w.observe(_refresh_output, names="value")

_refresh_output()

# Apply button to save into variables
apply_btn = widgets.Button(description="Apply to variables", button_style="success")
status = widgets.HTML()


def _on_apply_clicked(_):
    global max_n_layers, n_embd, mlp_ratio, block_size, max_n_heads, config

    # Validate before save
    errs = []
    if w_n_embd_step.value <= 0:
        errs.append("Embedding step must be > 0")
    if w_n_embd_min.value > w_n_embd_max.value:
        errs.append("Embedding min cannot be greater than max")

    if w_mlp_step.value <= 0:
        errs.append("MLP ratio step must be > 0")
    if w_mlp_min.value > w_mlp_max.value:
        errs.append("MLP ratio min cannot be greater than max")

    if w_bs_step.value <= 0:
        errs.append("Block size step must be > 0")
    if w_bs_min.value > w_bs_max.value:
        errs.append("Block size min cannot be greater than max")

    if errs:
        status.value = "<span style='color:red'>" + "; ".join(errs) + "</span>"
        return

    max_n_layers = w_max_n_layers.value
    max_n_heads = w_max_n_heads.value
    n_embd = {
        "min": w_n_embd_min.value,
        "max": w_n_embd_max.value,
        "step": w_n_embd_step.value,
    }
    mlp_ratio = {
        "min": w_mlp_min.value,
        "max": w_mlp_max.value,
        "step": w_mlp_step.value,
    }
    block_size = {
        "min": w_bs_min.value,
        "max": w_bs_max.value,
        "step": w_bs_step.value,
    }
    config = dict(
        max_n_layers=max_n_layers,
        n_embd=n_embd,
        mlp_ratio=mlp_ratio,
        block_size=block_size,
        max_n_heads=max_n_heads,
    )

    status.value = (
        "<span style='color:green'>Saved: max_n_layers, n_embd {min,max,step}, mlp_ratio {min,max,step}, "
        "block_size {min,max,step}, max_n_heads, and config</span>"
    )


apply_btn.on_click(_on_apply_clicked)

display(ui, out, apply_btn, status)


## Check the Search Space Configuration

In [ ]:
# Example: use the saved variables
try:
    print("max_n_layers:", max_n_layers)
    print("n_embd:", n_embd)
    print("max_n_heads:", max_n_heads)
    print("\nConfig dict:", config)
except NameError:
    print("Variables not set yet. Run the previous cell and click 'Apply to variables'.")

## Inspect Search Space Size -- Homogeneous Architecture

Inspect the size of the search space -- only considering variations on the software side

In [ ]:
# Import the search_space.py next to this notebook's folder and build a space from the UI variables
import sys
from pathlib import Path

# Ensure the nsga_search directory (parent of this notebook) is on sys.path
nb_dir = Path().resolve()              # .../nsga_search/notebooks
src_dir = nb_dir.parent                # .../nsga_search
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

num_configs = 0 

for n_embdin in range(n_embd["min"], n_embd["max"] + 1, n_embd["step"]):
    for n_heads in range(1, max_n_heads + 1):
        if n_embdin % n_heads == 0:
            num_configs += (mlp_ratio["max"] - mlp_ratio["min"] + 1) * ((block_size["max"] - block_size["min"]) // block_size["step"] + 1)
            
num_configs *= max_n_layers  # arbitrary number of layers, each can be any of the configurations

print(f"Number of possible configurations in the search space: {num_configs}")


### Mapping to Transformer Accelerator Hardware 

We designed a custom transformer accelerator which can run Transformer models defined above. 

![](images/transformer_hw.png)

Figure (a) decribed the overall architecture of the systolic array based transformer accelerator, Figure (b) describes the structure with in a single core. 

We can explore this hardware search space with the parameters and constraints defined in the table below:

| Parameter | Description | Range |
|:--:|:--|:--|
| $\text{n\_row}$ | Number of rows in the systolic array | keep the same number as the head size | 
| $\text{n\_col}$ | Number of columns in the systolic array | [1,64] | 
| $\text{n\_mac}$ | Number of mac units  | [4,8,16,32] | 
| $\text{wmem\_depth}$ | Block size | [128,256,512,768,1024,1536,2048,2560,3072,3584,4096] | 
| $\text{kv\_cache\_depth}$ | Layers | [128,256,512,768,1024,1536,2048,2560,3072,3584,4096] |

The computation for each head will be mapped onto one row of the systolic array by design. So we have n_row always equal to n_head.

To adapt to the dataflow decribed in the figure, we have the following constraint for the n_mac and n_cols: 

<p align="center"> ( n_embd / n_head / n_col ) % n_mac == 0 </p>
<p align="center"> block_size % n_mac == 0</p>

![](images/n_cols_description.png)

Based on the hardware search space description here, the hardware-software co-design space can be enlarged to the following size.

In [ ]:
max_n_col = 64
max_n_mac = 32

num_hw_sw_configs = 0   
num_hw_sw_raw_configs = 0

def is_feasible(n_embd, n_heads, n_cols, n_macs, max_context_length) -> bool:
    """Check if the suggestion is feasible."""

    if n_embd % n_heads != 0:
        # print(f"n_embd {n_embd} is not divisible by n_heads {n_heads}. Reject suggestion.")
        return False
    
    head_dim = int(n_embd/n_heads)
    if head_dim % n_cols != 0:
        # print(f"head_dim {head_dim} is not divisible by n_cols {n_cols}. Reject suggestion.")
        return False
    
    core_dim = int(head_dim/n_cols)
    if core_dim % n_macs != 0:
        # print(f"core_dim {core_dim} is not divisible by mac_num {mac_num}. Reject suggestion.")
        return False
    
    if max_context_length % n_cols != 0:
    #   print(f"max_context_length {max_context_length} is not divisible by n_cols {n_cols}. Reject suggestion.")
      return False

    return True

for block_size_val in range(block_size["min"], block_size["max"] + 1, block_size["step"]):
    for n_embdin in range(n_embd["min"], n_embd["max"] + 1, n_embd["step"]):
        for n_heads in range(1, max_n_heads + 1):
            if n_embdin % n_heads != 0:
                continue
            for n_col in range(1, max_n_col + 1):
                for n_mac in range(1, max_n_mac + 1):
                    num_hw_sw_raw_configs += (mlp_ratio["max"] - mlp_ratio["min"] + 1)
                    if is_feasible(n_embdin, n_heads, n_col, n_mac, max_context_length=block_size_val):
                        num_hw_sw_configs += (mlp_ratio["max"] - mlp_ratio["min"] + 1) 

print(f"Number of hardware-software co-design configurations in the search space: {num_hw_sw_raw_configs}. After filtering infeasible ones: {num_hw_sw_configs}.")
print(f"The HW/SW co-design search space is {num_hw_sw_configs/num_configs:.2f}x larger than the SW-only search space.")


## HW Evaluation Methods

Mapping SRAM configuration for each PE:

Below is a compact summary of the SRAM blocks mapped per PE, with their width and depth expressions (using the search-space symbols):

| Name | Width | Depth |
|:--|:--:|:--:|
| KV cache | `n_mac * 8` | `block_size * n_embd * 2 / n_row / n_col / n_mac` |
| WMEM     | `n_mac * 8` | `(mlp_ratio * n_embd**2 + 2 * mr * n_embd) / n_row / n_col / n_mac` |

For convenience, we round up the sram depth to the nearest value in this list: `[128,256,512,768,1024,1536,2048,2560,3072,3584,4096]`

The way we did HW sweeping is by sweeping the configurations for each core(PE) in `sky130` pdk and then aggregate the hw metrics analytically. We use the hw metrics data after the synthesis step.

The folling figure describes the flow to get hardware metrics(PPA)

![](images/ppa_flow.png)

Let's start with sweeping the search space for one core(PE).


#### Run from this notebook

In [ ]:
from os import path
import sys
import numpy as np
import matplotlib.pyplot as plt

# sys.path.append(path.join(path.dirname(__file__), '.'))
from pyppa import PPARunner
from pyppa.tools import Yosys, OpenROAD, Iverilog
from platforms.sky130hd.config import SKY130HD_PLATFORM_CONFIG

print(sys.path)

# Initialize a PPA runner
ppa_runner = PPARunner(
	# Design name can be anything
	design_name="core_top",
	# Define the tools to be used here
	tools={
		'verilog_sim_tool': Iverilog(scripts_dir=path.join('scripts', 'iverilog')),
		'synth_tool': Yosys(scripts_dir=path.join('scripts', 'synth')),
		'ppa_tool': OpenROAD(scripts_dir=path.join('scripts', 'ppa'))
	},
	# The global flow configuration that applies to all jobs
	global_flow_config={
		# Source Verilog files.
		'VERILOG_FILES': [
            path.join('verilog', 'sys_defs.svh'),
            path.join('verilog', 'core/core_acc.v'),
            path.join('verilog', 'core/core_buf.v'),
            path.join('verilog', 'core/core_ctrl.v'),
            path.join('verilog', 'core/core_mac.v'),
            path.join('verilog', 'core/core_mem.v'),
            path.join('verilog', 'core/core_quant.v'),
            path.join('verilog', 'core/core_rc.v'),
            path.join('verilog', 'core/core_top.v'),
            path.join('verilog', 'util/pe.v'),
            path.join('verilog', 'util/sram_sky130.v'),
            path.join('verilog', 'util/align.v'),
        	path.join('verilog', 'util/sky130_sram_stub.v')
		],
		# The constraint SDC file path.
		'SDC_FILE': path.join('verilog', 'constraint.sdc')
	}
)

# Set the platform configuration
ppa_runner.set_platform(SKY130HD_PLATFORM_CONFIG)

# Add a new sweep PPA job. This job sweeps a range of flow configurations and hyperparameters
ppa_runner.add_job({
	# Name of the Verilog module to run the PPA job on
	'module_name': 'core_top',
	'mode': 'sweep',
	# This dictionary sets the flow configuration options for this job only. The options set here are appended to the global_flow_config options.
	# To use multiple sets of values (all of which will be swept), use a dictionary. See the option `ABC_AREA` below and `clk_period` in hyperparameters for more information.
	'flow_config': {
		# If this option is set to True, Verilgo simulations will be run using the verilog_sim_tool set above. In this example, IVerilog is used.
		'RUN_VERILOG_SIM': True,
		'RUN_VERILOG_SIM': True,
		# This sets the netlist used for running the Verilog simulations. In this case, the postsynthesis Verilog netlist will be used.
		'VERILOG_SIM_TYPE': 'postsynth',
		# A list of the required testbench files. The design files are automatically included and need not be added here.
		'VERILOG_TESTBENCH_FILES': [
			path.join('verilog', 'core/core_top_tb.v')
		],
		# If this option is set to true, a VCD file dumped from the simulations will be used to get more accurate power estimates.
		'USE_STA_VCD': True,
		# The name of the VCD file dumped. By default it is set to `module_name.vcd`
		'VERILOG_VCD_NAME': 'core_top.vcd'
		,'ADDITIONAL_LIB_FILES': [
            path.join('verilog', 'util/lib/sky130_sram_0kbytes_1rw_32x128_32.lib'),
			path.join('verilog', 'util/lib/sky130_sram_2kbytes_1rw_32x512_32.lib')
        ]
		,'ADDITIONAL_LEFS': [
			path.join('verilog', 'util/lef/sky130_sram_0kbytes_1rw_32x128_32.lef'),
			path.join('verilog', 'util/lef/sky130_sram_2kbytes_1rw_32x512_32.lef')
        ]
        ,'BLOCKS': [
			'sky130_sram_0kbytes_1rw_32x128_32',
			'sky130_sram_2kbytes_1rw_32x512_32'
		]
		,'ABC_MAX_FANOUT': {
			'start': 40,
			'end': 40,
			'step': 4
		},
		'ABC_MAP_EFFORT': {
			'values': [0.5]
		},
		'ABC_AREC_EFFORT': {
            'values': [0.5]
		},
		'SYNTH_HIERARCHICAL': {
			'values': [False]
		}
	},
	# Hyperparameters are used defined parameters that can be inserted in the source files using the Mako templating syntax. See https://www.makotemplates.org/ for more information.
	# The simplest way is to write ${clk_period} in any source files (Verilog, Verilog testbench file, or constraint.sdc) to replace the value with the parameters set.
	# See the constraint.sdc file for example syntax usage.
	# The hyperparameters are swept along with the flow config, and all possible combinations of the options will be swept.
	'hyperparameters': {
		# The dictionary below defines a sweep for the `clk_period` hyperparameter. All values of clk_period, starting at `10` and going upto `100` will be swept with a step of 10. i.e., 10, 20, ..., 100.
		# This hyperparameter is used to set the clock period in the constraint.sdc and the verilog testbench.
		'clk_period': {
			'values': [3,4,5,6]
		},
		'mac_num': {
			'values': [4,8,16,32]
		},
		'wmem_depth': {
			'values': [128,256,512,768,1024,1536,2048,2560,3072,3584,4096]
		},
		'kv_cache_depth': {
			'values': [128,256,512,768,1024,1536,2048,2560,3072,3584,4096]
		}
	}
})
import time
start_time = time.time()

ppa_runner.run_all_jobs()

end_time = time.time()
total_time = end_time - start_time

clk_period = []
power = []
area = []

# Reading the PPA results in Python
for job_run in ppa_runner.job_runs:
	# The `job_runs` variable contains the PPA results for each job
	for ppa_run in job_run['ppa_runs']:
		# Each job run contains multiple "PPA Runs", each of which represents a particular configuration that was swept
		clk_period.append(ppa_run['ppa_stats']['sta']['clk']['clk_period'])
		power.append(ppa_run['ppa_stats']['power_report']['total']['total_power'])
		area.append(ppa_run['synth_stats']['module_area'])
		
		print("\n")
		print(f"Results for run #{ppa_run['run_number']}:")
		# print(f"ABC max fanout: {ppa_run['flow_config']['ABC_MAX_FANOUT']}")  # This is the value of the ABC_MAX_FANOUT option
		# print(f"ABC map effort: {ppa_run['flow_config']['ABC_MAP_EFFORT']}")  # This is the value of the ABC_MAP_EFFORT option
		print(f"wmem_depth: {ppa_run['hyperparameters']['wmem_depth']}, mac_num: {ppa_run['hyperparameters']['mac_num']}, clk_period: {ppa_run['hyperparameters']['clk_period']}")
		print(f"PPA stats: {ppa_run['ppa_stats']['power_report']['total']['total_power']} W")
		print(f"STA report: slack {ppa_run['ppa_stats']['sta']['clk']['clk_slack']} period {ppa_run['ppa_stats']['sta']['clk']['clk_period']} total {ppa_run['ppa_stats']['sta']['clk']['clk_period']+ppa_run['ppa_stats']['sta']['clk']['clk_slack']}")
		print(f"Total cells={ppa_run['synth_stats']['num_cells']}, Area={ppa_run['synth_stats']['module_area']}, Seq/Comb cells = {ppa_run['ppa_stats']['num_sequential_cells']}/{ppa_run['ppa_stats']['num_combinational_cells']};")
		print("=========================================================")

print(clk_period)
print(power)
print(area)
print(f"Total time taken for PPA runs: {total_time:.2f} seconds")



The completion time for this sweeping is over 12h. To save time, I put the extracted data in the ref_data folder `ref_data/core_top_sweep_extracted_data.csv`. 

### Sweep SW-HW design points

We also sweep the software design points with decoder-only architecture with size of 100M. We provided the data in the ref_data folder `ref_data/sw_sweep_data.csv`

The following figure presents the sweeping process.

![](images/sweep_flow.png)

We will demo the sw and hw data matching process next.

In [ ]:
# iterate through the sw swweep data and map it to sw_hw data
import pandas as pd
import numpy as np
import math

# Load data again
file_path = './ref_data/core_top_sweep_extracted_data.csv'
data = pd.read_csv(file_path)

# Creating a nested dictionary-based database
database = {}

def is_feasible(n_embd, n_heads, n_cols, n_macs, max_context_length) -> bool:
    """Check if the suggestion is feasible."""
    if n_embd % n_heads != 0:
        return False
    head_dim = int(n_embd/n_heads)
    if head_dim % n_cols != 0:
        return False
    core_dim = int(head_dim/n_cols)
    if core_dim % n_macs != 0:
        return False
    if max_context_length % n_cols != 0:
      return False
    return True

def get_cache_depth(n_model, n_heads, n_cols, max_context_length, n_macs) -> int:
    """Get the cache depth based on the suggestion."""
    raw_cache_depth = int(2 * n_model * max_context_length / n_macs / n_cols / n_heads)
    if raw_cache_depth < 128:
        return 128
    elif raw_cache_depth < 256:
        return 256
    else:
        return int(math.ceil(raw_cache_depth / 512) * 512)

def get_wmem_depth(n_model, n_heads, n_cols, n_macs, ffn_ratio) -> int:
    """Get the wmem depth based on the suggestion."""
    raw_wmem_depth = int((4 + 0*ffn_ratio)*(n_model * n_model)/ n_heads / n_cols / n_macs)
    if raw_wmem_depth < 32:
        return 32
    elif raw_wmem_depth < 64:
        return 64
    elif raw_wmem_depth < 128:
        return 128
    else :
        return int(math.ceil(raw_wmem_depth / 512) * 512)
    
def get_TTFT(clk_period, n_model, mac_num, n_heads, n_cols, max_context_length, sequence_length=256 ,n_layers=1, ffn_ratio=4, softmax_choice='SOFTMAX', activation_choice='RELU'):
    """
    Calculate the time to first token (TTFT) based on the design parameters.
    """
    # coerce inputs to numeric types (some callers pass strings)
    n_model = float(n_model)
    mac_num = float(mac_num)
    n_heads = float(n_heads)
    n_cols = float(n_cols)
    max_context_length = float(max_context_length)
    sequence_length = float(sequence_length)
    n_layers = float(n_layers)
    ffn_ratio = float(ffn_ratio)
    # Calculate the token delay for GEMM ops
    TTFT = (4.0 * n_model * n_model + 2.0 * ffn_ratio * n_model * n_model) * 1e-9 * clk_period / gbus_width # load and store delay
    # Number of cycles = Total MACs / (Number of MAC units)
    num_loading_cycles = (4*n_model*n_model*sequence_length + 2*max_context_length*max_context_length*n_model + 2*ffn_ratio*n_model*n_model*sequence_length) / (n_heads*n_cols*mac_num)
    TTFT += num_loading_cycles * clk_period * 1e-9 # seconds
    # add 2 residual delay
    TTFT += 2 * n_model * sequence_length * 1e-9 * clk_period # residual add delay
    # add memory loading delay

    # activation delay is negligible

    TTFT /= 0.9 # assuming 90% efficiency

    TTFT *= n_layers

    return TTFT # convert to s

def get_TPOT(clk_period, n_model, mac_num, n_heads, n_cols, max_context_length, sequence_length=256 ,n_layers=1, ffn_ratio=4, softmax_choice='SOFTMAX', activation_choice='RELU'):
    """
    Calculate the time per output token (TPOT) based on the design parameters.
    """
    # Calculate the token delay
    # for GEMM ops

    # coerce numeric inputs
    n_model = float(n_model)
    mac_num = float(mac_num)
    n_heads = float(n_heads)
    n_cols = float(n_cols)
    max_context_length = float(max_context_length)
    sequence_length = float(sequence_length)
    n_layers = float(n_layers)
    ffn_ratio = float(ffn_ratio)

    TPOT = (4.0 * n_model * n_model + 2.0 * ffn_ratio * n_model * n_model) * 1e-9 * clk_period # load and store delay

    # Number of cycles = Total MACs / (Number of MAC units)
    num_loading_cycles = (4*n_model*n_model + 2*sequence_length*sequence_length*n_model + 2*ffn_ratio*n_model*n_model) / (n_heads*n_cols*mac_num)
    TPOT += num_loading_cycles * clk_period * 1e-9 # seconds

    # add 2 residual delay
    TPOT += 2 * n_model * 1e-9 * clk_period # residual add delay

    # add memory loading delay
    TPOT /= 0.9 # assuming 90% efficiency
    TPOT *= n_layers
    return TPOT # convert to s

# Populating the database with configuration as keys and relevant metrics as values
for _, row in data.iterrows():
    key = (row['MAC NUM'], row['Wmem Depth'], row['Cache Depth'], row['Clock Period (ns) Entered'])
    database[key] = {
        'power': row['Power (W)'],
        'slack': row['Clock Slack (ns)'],
        'clk_min_period': row['Clock_Min_Period'],
        'area': row['Area (um^2)']
        # Additional metrics can be added here as needed
    }

df = pd.read_csv("./ref_data/sw_sweep_data.csv")

n_cols_range = np.arange(1, 99).tolist()
mac_num = [4,8,16,32]
clk_periods = [3,4,5,6]
sequence_length = 256

total_count = 0
feasible_configs = []

for idx, row in df.iterrows():
    n_model = row['n_embd']
    n_heads = row['n_head']
    max_context_length = row['block_size']
    val_loss = row['best_val_loss']
    n_layers = row['n_layer']
    ffn_ratio = row['mlp_expansion_factor']
    for clk_period in clk_periods:
        for n_cols in n_cols_range:
            for n_macs in mac_num:
                if is_feasible(n_model, n_heads, n_cols, n_macs, max_context_length):
                    # Calculate wmem_depth and cache_depth
                    wmem_depth = get_wmem_depth(n_model, n_heads, n_cols, n_macs, ffn_ratio)
                    cache_depth = get_cache_depth(n_model, n_heads, n_cols, max_context_length, n_macs)

                    # Retrieve core power and area from the database
                    core_power = database.get((n_macs, wmem_depth, cache_depth, clk_period), {}).get('power', 'N/A') 
                    core_area = database.get((n_macs, wmem_depth, cache_depth, clk_period), {}).get('area', 'N/A')
                    clk_min_period = database.get((n_macs, wmem_depth, cache_depth, clk_period), {}).get('clk_min_period', 'N/A')
                    slack = database.get((n_macs, wmem_depth, cache_depth, clk_period), {}).get('slack', 'N/A')

                    # calculate perplexity
                    perplexity = math.exp(val_loss) if val_loss != 'N/A' else 'N/A'

                    if core_power != 'N/A' and core_area != 'N/A' and clk_min_period != 'N/A':
                        total_count += 1
                        gbus_width = n_macs * 8
                        ttft = get_TTFT(clk_min_period, n_model, n_macs, n_heads, n_cols, max_context_length, ffn_ratio=ffn_ratio, n_layers=n_layers, sequence_length=sequence_length)
                        tpot = get_TPOT(clk_min_period, n_model, n_macs, n_heads, n_cols, max_context_length, ffn_ratio=ffn_ratio, n_layers=n_layers, sequence_length=sequence_length)
                        throughput = 1/(tpot) if tpot != 0 else 'N/A'

                        total_area = core_area * n_heads * n_cols
                        total_power = core_power * n_heads * n_cols
                        feasible_configs.append({
                            'n_embd': n_model,
                            'n_head': n_heads,
                            'block_size': max_context_length,
                            'n_layer': n_layers,
                            'mlp_expansion_factor': ffn_ratio,
                            'n_cols': n_cols,
                            'MAC NUM': n_macs,
                            'Wmem Depth': wmem_depth,
                            'Cache Depth': cache_depth,
                            'best_val_loss': val_loss,
                            'perplexity': perplexity,
                            'Power (W)': total_power,
                            'Area (um^2)': total_area,
                            'Clock Period (ns) Entered': clk_period,
                            'Clock_Min_Period': clk_min_period,
                            'F_max (MHz)': 1000/clk_min_period if clk_min_period != 0 else 'N/A',
                            'Clock Slack (ns)': slack,
                            'TTFT (ms)': ttft * 1000,
                            'TPOT (ms)': tpot * 1000,
                            'Throughput (tokens/s)': throughput,
                            'Energy per token (mJ/token)': total_power / throughput * 1000,
                            'EDP (mJ * ms)': (total_power/throughput/1e6) * ttft,
                            'EADP (mJ * ms * um^2)': (total_power/throughput/1e6) * ttft * total_area
                        })

print(f"Total number of feasible configurations: {total_count}")
feasible_df = pd.DataFrame(feasible_configs)

# Save to CSV
feasible_csv_path = './ref_data/sweep_data_sw_hw.csv'
feasible_df.to_csv(feasible_csv_path, index=True)



scatter plots with the three main metrics: val_loss, TTFT, Energy per token

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Reload the newest dataset
file_path = "./ref_data/sweep_data_sw_hw.csv"
df = pd.read_csv(file_path)

# Clean column names
df = df.rename(columns=lambda x: x.strip().replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_"))

# Normalize EDP and EADP for comparison
df["EDP_norm"] = (df["EDP_mJ_*_ms"] - df["EDP_mJ_*_ms"].min()) / (df["EDP_mJ_*_ms"].max() - df["EDP_mJ_*_ms"].min())
df["EADP_norm"] = (df["EADP_mJ_*_ms_*_um^2"] - df["EADP_mJ_*_ms_*_um^2"].min()) / (df["EADP_mJ_*_ms_*_um^2"].max() - df["EADP_mJ_*_ms_*_um^2"].min())

# Function for Pareto frontier
def pareto_frontier(df, x, y):
    points = df[[x, y]].values
    points = points[np.argsort(points[:, 0])]
    frontier = []
    min_y = np.inf
    for px, py in points:
        if py < min_y:
            frontier.append((px, py))
            min_y = py
    return np.array(frontier)

# Metrics to plot separately
metrics = {
    # "Power_W": "Power (W)",
    # "Area_um^2": "Area (um^2)",
    "TTFT_ms": "TTFT (ms)",
    # "TPOT_ms": "TPOT (ms)",
    "Energy_per_token_mJ_token": "Energy per Token (mJ/token)",
    # "EDP_norm": "Normalized EDP",
    # "EADP_norm": "Normalized EADP"
}

# Generate 5 separate figures
for metric, label in metrics.items():
    pareto = pareto_frontier(df, metric, "best_val_loss")
    
    plt.figure(figsize=(8,6))
    plt.scatter(df[metric], df["best_val_loss"], s=5, alpha=0.3, label="All Points")
    plt.plot(pareto[:,0], pareto[:,1], c="red", linewidth=2, label="Pareto Frontier")
    plt.xlabel(label)
    plt.ylabel("Best Val Loss")
    plt.title(f"Pareto Frontier: {label} vs Validation Loss")
    plt.grid(True, which="both", ls="--", linewidth=0.5)
    plt.legend()
    plt.show()
    # plt.savefig(f"../plots/pareto/pareto_{metric}.png", dpi=500)

# plot best_val_loss as the color map on TTFT vs Energy per token scatter plot
pareto = pareto_frontier(df, "TTFT_ms", "Energy_per_token_mJ_token")
plt.figure(figsize=(8,6))
plt.scatter(df["TTFT_ms"], df["Energy_per_token_mJ_token"], s=5, alpha=0.3, label="All Points")
plt.plot(pareto[:,0], pareto[:,1], c="red", linewidth=2, label="Pareto Frontier")
plt.xlabel("TTFT (ms)")
plt.ylabel("Energy per Token (mJ/token)")
plt.title("Energy per Token vs TTFT")
plt.grid(True, which="both", ls="--", linewidth=0.5)
plt.legend()
plt.show()

## Beyond Traditional Transformer Architecture

The above exploration is based on gpt2-like transformer architecture, while the modern edge scale language models have adopted architectural variation to boost model effiency.

### From Homogeneous to Heterogeneous Architecture

With heterogeneous architecture, each layer can have different parameters such as number of heads and mlp size. Now if we consider the paramter range we set before, we could have a much larger search space by allowing heterogeneous architecture. As we can see from the following calculation, the total number of feasible designs go up to astronomical figures.

![](images/hetero.png)

In [15]:
# Build specs from interactive values and instantiate the search space
try:
    # Globals belong in globals_spec
    gspec = {
        "d_model": {"type": "int", "low": n_embd["min"], "high": n_embd["max"], "step": n_embd["step"]},
    }
    # Per-layer fields go into layer_spec
    lspec = {
        "n_heads": {"type": "int", "low": 1, "high": max_n_heads, "step": 1},
        "block_size": {"type": "int", "low": block_size["min"], "high": block_size["max"], "step": block_size["step"]},
        "mlp_ratio": {"type": "int", "low": mlp_ratio["min"], "high": mlp_ratio["max"], "step": mlp_ratio["step"]},
        "attn_type": {"type": "cat", "choices": ["mha"]},
    }
    print("Search space ready. L_max =", max_n_layers)
except NameError:
    print("Variables not set yet. Run the interactive cell above and click 'Apply to variables'.")
    
num_configs_het = 0 

for n_embdin in range(n_embd["min"], n_embd["max"] + 1, n_embd["step"]):
    for n_heads in range(1, max_n_heads + 1):
        if n_embdin % n_heads == 0:
            num_configs_het += (mlp_ratio["max"] - mlp_ratio["min"] + 1) * ((block_size["max"] - block_size["min"]) // block_size["step"] + 1)
            
num_configs_het = num_configs_het ** max_n_layers  # Each layer can be any of the configurations

# use scientific notation if too large
print(f"Number of possible configurations in the search space: {num_configs_het:.2e} {num_configs_het // num_configs :.2e}x larger than the homogeneous space")

Search space ready. L_max = 12
Number of possible configurations in the search space: 1.68e+48 1.34e+43x larger than the homogeneous space


###  Introducing Infinite Attention

Original MHA has this disadvantage that the n_head and head_dim are tight to the number of embeddings, this will restrict us from exploring possibilities of head pruning and different layer sizes。

We introduce Infinite Attention, in which head_dim does not have to be n_embd/n_head and v_head_dim and qk_head_dim can be different, enabling us to explore a broader search space.

![](images/infinite_attention.png)

The figure above shows the structure of Infiite Attention.

The searchable parameters are documented on the following table:

#### Global Parameter Space

| Parameter | Type | Range / Choices | Step |
|:--|:--|:--|:--|
| $\text{n\_embd}$ | int | fixed 768 | — |
| $\text{block\_size}$ | int | fixed 512 | — |
| $\text{use\_concat\_heads}$ | categorical | {true, false} | — |

#### Per-Layer Parameter Space

| Parameter | Type | Range / Choices | Step |
|:--|:--|:--|:--|
| $\text{n\_h}$ | int | 1 – 16 | 1 |
| $\text{n\_kv}$ | int | 1 – 16 | 1 |
| $\text{mlp\_size}$ | int | 256 – 4096 | 256 |
| $\text{qk\_dim}$ | int | 32 – 512 | 32 |
| $\text{v\_dim}$ | int | 32 – 512 | 32 |
| $\text{n\_cproj}$ | int | 1 – 4 | 1 |
| $\text{use\_concat\_head}$ | bool | True/False | - |
| $\text{attention\_variant}$ | categorical | {infinite, identity} | - |

**identity means skipping the attention module for this layer**

## Multi-Objective Optimization of Transformer SW Architecture Enabled by NSGA-II Architecture

As mentioned before, now the size of the entire search space has gone up to a astronomical figure. Sweeping through each feasible design point becomes unrealistic. We need a smarter algorithm to help us explore new design points efficiently. Here we decide to implement NSGA2, a multi-objective variant of the genetic algorithm for the optimization strategy. 

![](images/nsga_framework.png)

We run the algorithm over our Infinite Attention architecture based heterogeneous search space.

![](images/exp_settings.png)


In [ ]:
# to run this experiment, follow the readme in the nsga_experiments folder from reallmasic repo
# the experiment setup can be configured
# the experiment is exposed to be run with a group of gpu machines with ssh ips defined in a yaml file

# clone the repo first
!git clone https://github.com/ReaLLMASIC/ReaLLM-Forge.git

# install the conda environment
!cd ReaLLM-Forge && bash dev_env_setup_scripts/00-setup-conda.sh

# install the requirements
!pip install -r requirements_gpu.txt

# fecth and tokenize the minipile dataset
!cd data/minipile && bash get_dataset.sh && python prepare.py -t input.txt 

# run the nsga experiment with the generated search space config
!cd ../../optimization_and_search/nsga_search  

# follow the README in the nsga_experiments folder and run the experiment bash
!bash run_from_scratch.bash




The completion time of this experiment on 8 h100 gpus is about 1.5 day. The results for this experiment is presented below.

![](images/exp_results.png)

Preliminary results show our models advancing toward the accuracy–size Pareto front, outperforming GPT-2 small.